In [ ]:
import os
import time
import re
from astroquery.simbad import Simbad
from astroquery.skyview import SkyView
import astropy.units as u
from astropy.coordinates import SkyCoord
import numpy as np
import math

# --- 藏宝图列表 (梅西耶 + 科德韦尔 + 赫歇尔400) ---
MESSIER_OBJECTS = [f"M{i}" for i in range(1, 111)]
CALDWELL_OBJECTS = ["NGC 188", "NGC 40", "NGC 4236", "NGC 7023", "NGC 381", "NGC 6543", "IC 418", "NGC 559", "Sh2-155", "NGC 663","NGC 7635", "NGC 6946", "NGC 457", "NGC 869", "NGC 884", "NGC 281", "NGC 147", "NGC 185", "IC 5146", "NGC 7331","NGC 2403", "NGC 7662", "NGC 891", "NGC 1232", "NGC 253", "NGC 4631", "NGC 6992", "NGC 6960", "NGC 246", "NGC 278","NGC 7293", "NGC 3521", "NGC 7814", "NGC 2841", "NGC 3115", "NGC 4449", "NGC 4736", "NGC 5055", "NGC 5866", "NGC 3628","NGC 4565", "NGC 3077", "NGC 2903", "NGC 2683", "NGC 5907", "NGC 3184", "NGC 4244", "NGC 4490", "NGC 5005", "NGC 5195","NGC 4656", "NGC 3626", "NGC 7479", "NGC 2859", "NGC 2976", "NGC 488", "NGC 4088", "NGC 6503", "NGC 5248", "NGC 3198","NGC 6888", "NGC 7000", "NGC 6210", "NGC 6826", "NGC 2359", "NGC 55", "IC 1613", "NGC 247", "NGC 300", "NGC 598","NGC 24", "NGC 1313", "NGC 1560", "NGC 1851", "NGC 2244", "NGC 2360", "NGC 2451", "NGC 2477", "NGC 2516", "NGC 2547","NGC 3132", "NGC 3195", "NGC 3201", "NGC 3242", "NGC 3372", "NGC 3532", "NGC 3766", "IC 2602", "NGC 4103", "NGC 4372","NGC 4609", "NGC 4755", "NGC 4945", "NGC 5128", "NGC 5139", "NGC 5286", "NGC 5662", "NGC 6025", "NGC 6067", "NGC 6087","NGC 6124", "NGC 6193", "NGC 6231", "NGC 6352", "NGC 6397", "NGC 6541", "NGC 6752"]
HERSCHEL_400 = ["NGC 188", "NGC 225", "NGC 278", "NGC 281", "NGC 381", "NGC 436", "NGC 457", "NGC 559", "NGC 609", "NGC 637", "NGC 654", "NGC 743", "NGC 771", "NGC 869", "NGC 884", "NGC 891", "NGC 1023", "NGC 1027", "NGC 1055", "NGC 1087", "NGC 1161", "NGC 1232", "NGC 1245", "NGC 1332", "NGC 1342", "NGC 1491", "NGC 1499", "NGC 1501", "NGC 1502", "NGC 1514", "NGC 1528", "NGC 1535", "NGC 1545", "NGC 1555", "NGC 1579", "NGC 1624", "NGC 1647", "NGC 1664", "NGC 1778", "NGC 1807", "NGC 1817", "NGC 1857", "NGC 1907", "NGC 1931", "NGC 1981", "NGC 2022", "NGC 2024", "NGC 2129", "NGC 2158", "NGC 2169", "NGC 2174", "NGC 2186", "NGC 2194", "NGC 2204", "NGC 2237", "NGC 2244", "NGC 2261", "NGC 2264", "NGC 2266", "NGC 2276", "NGC 2281", "NGC 2301", "NGC 2304", "NGC 2309", "NGC 2331", "NGC 2346", "NGC 2359", "NGC 2360", "NGC 2362", "NGC 2371", "NGC 2392", "NGC 2403", "NGC 2419", "NGC 2420", "NGC 2438", "NGC 2440", "NGC 2506", "NGC 2539", "NGC 2548", "NGC 2571", "NGC 2655", "NGC 2683", "NGC 2841", "NGC 2859", "NGC 2903", "NGC 2964", "NGC 2976", "NGC 3077", "NGC 3079", "NGC 3115", "NGC 3184", "NGC 3190", "NGC 3198", "NGC 3242", "NGC 3344", "NGC 3432", "NGC 3486", "NGC 3521", "NGC 3556", "NGC 3626", "NGC 3628", "NGC 3718", "NGC 3726", "NGC 4036", "NGC 4088", "NGC 4102", "NGC 4145", "NGC 4151", "NGC 4214", "NGC 4236", "NGC 4244", "NGC 4290", "NGC 4388", "NGC 4395", "NGC 4449", "NGC 4490", "NGC 4494", "NGC 4535", "NGC 4559", "NGC 4565", "NGC 4605", "NGC 4631", "NGC 4656", "NGC 4725", "NGC 4736", "NGC 4762", "NGC 4861", "NGC 5005", "NGC 5024", "NGC 5033", "NGC 5053", "NGC 5195", "NGC 5248", "NGC 5353", "NGC 5457", "NGC 5474", "NGC 5566", "NGC 5634", "NGC 5676", "NGC 5689", "NGC 5746", "NGC 5813", "NGC 5822", "NGC 5823", "NGC 5838", "NGC 5866", "NGC 5907", "NGC 5981", "NGC 5982", "NGC 5985", "NGC 6015", "NGC 6027", "NGC 6045", "NGC 6124", "NGC 6166", "NGC 6193", "NGC 6207", "NGC 6210", "NGC 6231", "NGC 6242", "NGC 6251", "NGC 6281", "NGC 6302", "NGC 6334", "NGC 6357", "NGC 6383", "NGC 6426", "NGC 6445", "NGC 6451", "NGC 6503", "NGC 6520", "NGC 6522", "NGC 6530", "NGC 6535", "NGC 6541", "NGC 6543", "NGC 6553", "NGC 6572", "NGC 6633", "NGC 6649", "NGC 6709", "NGC 6712", "NGC 6729", "NGC 6760", "NGC 6781", "NGC 6791", "NGC 6802", "NGC 6811", "NGC 6818", "NGC 6819", "NGC 6820", "NGC 6822", "NGC 6823", "NGC 6826", "NGC 6830", "NGC 6834", "NGC 6842", "NGC 6866", "NGC 6871", "NGC 6888", "NGC 6894", "NGC 6905", "NGC 6910", "NGC 6934", "NGC 6939", "NGC 6940", "NGC 6946", "NGC 6951", "NGC 6960", "NGC 6992", "NGC 7000", "NGC 7006", "NGC 7008", "NGC 7009", "NGC 7023", "NGC 7027", "NGC 7044", "NGC 7048", "NGC 7062", "NGC 7092", "NGC 7129", "NGC 7139", "NGC 7142", "NGC 7160", "NGC 7209", "NGC 7235", "NGC 7243", "NGC 7245", "NGC 7261", "NGC 7293", "NGC 7331", "NGC 7332", "NGC 7380", "NGC 7419", "NGC 7439", "NGC 7457", "NGC 7479", "NGC 7510", "NGC 7538", "NGC 7635", "NGC 7662", "NGC 7686", "NGC 7762", "NGC 7772", "NGC 7789", "NGC 7814"]
MASTER_CATALOG = sorted(list(set(MESSIER_OBJECTS + CALDWELL_OBJECTS + HERSCHEL_400)))

# --- 参数设置 ---
IMAGE_SURVEY = 'DSS'
PIXELS = 4096
DEFAULT_FOV_ARCMIN = 45.0
MOSAIC_THRESHOLD_ARCMIN = 90.0
MOSAIC_TILE_FOV_ARCMIN = 45.0
MOSAIC_OVERLAP = 0.2
OUTPUT_DIR = "dso_images_4k_45arcmin_final"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def download_final_production_script_v4():
    print(f"--- 开始处理主目录，总计 {len(MASTER_CATALOG)} 个高质量天体 ---")
    print(f"--- 图像分辨率: {PIXELS}x{PIXELS}, 默认/瓦片视场: {DEFAULT_FOV_ARCMIN}' ---")
    
    custom_simbad = Simbad()
    # ==========================[ 次要修正 ]==========================
    # 更新过时的字段名 'ra(d)' -> 'ra', 'dec(d)' -> 'dec'
    # ================================================================
    custom_simbad.add_votable_fields('mesdiameter', 'ra', 'dec')

    for i, obj_name in enumerate(MASTER_CATALOG):
        print(f"\n[{i+1}/{len(MASTER_CATALOG)}] 正在处理: {obj_name}")
        
        try:
            result_table = custom_simbad.query_object(obj_name)
            if result_table is None: continue
            
            main_id = result_table['main_id'][0]
            safe_filename = re.sub(r'[\s/]', '_', main_id)
            
            major_axis = 0
            if 'mesd_diameter' in result_table.colnames and 'mesd_unit' in result_table.colnames:
                arcmin_diams = [d for d, u in zip(result_table["mesd_diameter"], result_table["mesd_unit"]) if u == 'arcmin']
                if arcmin_diams: major_axis = np.nanmax(arcmin_diams)
            
            if major_axis > MOSAIC_THRESHOLD_ARCMIN:
                print(f"  - 天体尺寸 ({major_axis:.2f}') 超过阈值，启动马赛克模式。")
                mosaic_dir = os.path.join(OUTPUT_DIR, f"{safe_filename}_mosaic")
                if os.path.exists(mosaic_dir):
                    print("  - 马赛克目录已存在，跳过。")
                    continue
                os.makedirs(mosaic_dir)

                center_ra, center_dec = result_table['ra'][0], result_table['dec'][0] # 使用修正后的字段名
                center_coord = SkyCoord(ra=center_ra*u.deg, dec=center_dec*u.deg, frame='icrs')
                
                step_size = MOSAIC_TILE_FOV_ARCMIN * (1 - MOSAIC_OVERLAP)
                grid_x = math.ceil(major_axis / step_size); grid_y = math.ceil(major_axis / step_size)

                print(f"  - 将创建 {grid_x} x {grid_y} 的网格。")
                
                for y in range(grid_y):
                    for x in range(grid_x):
                        ra_offset = (x - (grid_x - 1) / 2.0) * step_size
                        dec_offset = (y - (grid_y - 1) / 2.0) * step_size
                        
                        tile_ra = center_ra + (ra_offset / 60 / math.cos(center_coord.dec.radian))
                        tile_dec = center_dec + (dec_offset / 60)
                        
                        tile_coord_str = f"{tile_ra}d {tile_dec}d"
                        tile_path = os.path.join(mosaic_dir, f"tile_{y}_{x}.fits")

                        print(f"    - 下载瓦片 ({y}, {x}) ...")
                        try:
                            # ==========================[ 关键修正 ]==========================
                            # 同时提供 width 和 height 参数
                            # ================================================================
                            images = SkyView.get_images(position=tile_coord_str, survey=IMAGE_SURVEY, pixels=str(PIXELS), 
                                                        width=MOSAIC_TILE_FOV_ARCMIN*u.arcmin, 
                                                        height=MOSAIC_TILE_FOV_ARCMIN*u.arcmin)
                            if images and len(images[0]) > 0:
                                images[0][0].writeto(tile_path, overwrite=True)
                            time.sleep(1)
                        except Exception as tile_e:
                            print(f"      - 下载瓦片失败: {tile_e}")
            else:
                fits_path = os.path.join(OUTPUT_DIR, f"{safe_filename}.fits")
                if os.path.exists(fits_path):
                    print("  - 图片已存在，跳过。")
                    continue

                fov_arcmin = DEFAULT_FOV_ARCMIN
                if major_axis > 0:
                    fov_arcmin = max(DEFAULT_FOV_ARCMIN, major_axis * FOV_BUFFER_FACTOR)
                fov_arcmin = min(fov_arcmin, 120)

                print(f"  - 正在下载单张图片 (视场: {fov_arcmin:.2f}')...")
                # ==========================[ 关键修正 ]==========================
                # 同时提供 width 和 height 参数
                # ================================================================
                images = SkyView.get_images(position=obj_name, survey=IMAGE_SURVEY, pixels=str(PIXELS), 
                                            width=fov_arcmin*u.arcmin, 
                                            height=fov_arcmin*u.arcmin)
                if images and len(images[0]) > 0:
                    images[0][0].writeto(fits_path, overwrite=True)
                    print(f"  - 成功保存图片: {safe_filename}.fits")

        except Exception as e:
            print(f"  - 处理 {obj_name} 时发生严重错误: {e}")
        
        time.sleep(1)

    print("\n所有任务完成！")

if __name__ == "__main__":
    download_final_production_script_v4()

--- 开始处理主目录，总计 402 个高质量天体 ---
--- 图像分辨率: 4096x4096, 默认/瓦片视场: 45.0' ---

[1/402] 正在处理: IC 1613
  - 图片已存在，跳过。

[2/402] 正在处理: IC 2602
  - 图片已存在，跳过。

[3/402] 正在处理: IC 418
  - 图片已存在，跳过。

[4/402] 正在处理: IC 5146
  - 图片已存在，跳过。

[5/402] 正在处理: M1
  - 图片已存在，跳过。

[6/402] 正在处理: M10
  - 图片已存在，跳过。

[7/402] 正在处理: M100
  - 图片已存在，跳过。

[8/402] 正在处理: M101
  - 图片已存在，跳过。

[9/402] 正在处理: M102
  - 图片已存在，跳过。

[10/402] 正在处理: M103
  - 图片已存在，跳过。

[11/402] 正在处理: M104
  - 图片已存在，跳过。

[12/402] 正在处理: M105
  - 图片已存在，跳过。

[13/402] 正在处理: M106
  - 图片已存在，跳过。

[14/402] 正在处理: M107
  - 图片已存在，跳过。

[15/402] 正在处理: M108
  - 图片已存在，跳过。

[16/402] 正在处理: M109
  - 图片已存在，跳过。

[17/402] 正在处理: M11
  - 图片已存在，跳过。

[18/402] 正在处理: M110
  - 图片已存在，跳过。

[19/402] 正在处理: M12
  - 图片已存在，跳过。

[20/402] 正在处理: M13
  - 图片已存在，跳过。

[21/402] 正在处理: M14
  - 图片已存在，跳过。

[22/402] 正在处理: M15
  - 图片已存在，跳过。

[23/402] 正在处理: M16
  - 图片已存在，跳过。

[24/402] 正在处理: M17
  - 图片已存在，跳过。

[25/402] 正在处理: M18
  - 图片已存在，跳过。

[26/402] 正在处理: M19
  - 图片已存在，跳过。

[27/402] 正在处理: M2
  - 图片已存在，